### Hyperparameters

In [1]:
# Hyperparameters

seqlen = 64
#batch_size = 1024
sample_size = 10

### Code

In [2]:
import tensorflow.keras as K
import os.path

import sys, imp

sys.path.append("../src")
from artstat import util
import numpy as np

In [10]:
imp.reload(util)

<module 'artstat.util' from '/app/src/artstat/util.py'>

In [6]:
path_train = "/data/local/gallery/train"
path_test = "/data/local/gallery/test"

glove_dims = 300

glove = "/data/shared/glove/glove.6B.%dd.txt" % glove_dims

#glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"
#glove = 
#glove = "/home/pmilovanov/data/glove/glove.6B.100d.txt"
#glove = "/home/pmilovanov/data/glove/glove.6B.300d.txt"


In [7]:
path_train

'/data/local/gallery/train'

In [8]:
#vocab_file = "../vocab.txt"
vocab_file = "../vocab_lower.txt"

words, vocab = util.load_vocab(vocab_file, 10000)

In [11]:
emb_matrix = util.load_embeddings(vocab, glove_dims, glove)

print(len(vocab))
print(emb_matrix.shape)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 123: ordinal not in range(128)

In [11]:
emb_matrix


array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.25539  , -0.25723  ,  0.13169  , ..., -0.2329   , -0.12226  ,
         0.35499  ],
       [ 0.04656  ,  0.21318  , -0.0074364, ...,  0.0090611, -0.20989  ,
         0.053913 ],
       ...,
       [ 0.64235  , -0.22695  ,  0.48475  , ..., -0.25002  ,  0.57936  ,
        -0.24314  ],
       [-0.31695  ,  0.29755  , -0.36873  , ...,  0.29704  ,  0.52806  ,
        -0.57199  ],
       [ 0.063258 , -0.18871  , -0.10758  , ...,  0.36241  , -0.23525  ,
         0.23106  ]], dtype=float32)

In [12]:
imp.reload(util)
X, Xu = util.load_data(path_train, vocab, pad=seqlen, lowercase=True)

 42%|####2     | 9697/22860 [00:19<00:26, 495.18it/s]

KeyboardInterrupt: 

 42%|####2     | 9697/22860 [00:30<00:40, 323.21it/s]

In [ ]:
print("Total words:", len(X))
print("Unknown words: {:.2f}%".format(100 * sum(Xu[:,0]) / len(Xu)))

In [ ]:
from K.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, Concatenate
from K import Model, Input
import tensorflow as tf

In [ ]:
def sampling_layer(x):
    data, sample_indices = x
    def gather_per_batch(e):
        batch, indices = e
        indices = tf.cast(indices, tf.int32)
        #print("batch", type(batch), batch.dtype, batch.shape)
        #print("indices", type(indices), indices.dtype, indices.shape)
        return (tf.gather(batch, indices),0)
    return tf.map_fn(gather_per_batch, (data, sample_indices),
                    parallel_iterations=batch_size*2, swap_memory=False)[0]


def sampling_layer_gather_nd(x):
    data, sample_indices = x
    return tf.gather_nd(data, tf.cast(sample_indices, tf.int32))


In [ ]:
from K.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, \
    Concatenate, Lambda, Activation, Dropout
from keras import Model, Input

def make_model(*, seqlen, sample_size, emb_matrix, 
               lstm_sizes=[256, 256],
               dense_size=300, dense_layers=3,
               aux_dim=2, dropout_rate=0.1):    
    dim = emb_matrix.shape[1] + 1
    
    input_x = Input((seqlen,), dtype="int32", name="input_x")
    input_aux = Input((seqlen, aux_dim), dtype="float32", name="input_aux")
    input_sample_indices = Input((sample_size,2), dtype="int32", name="input_sample_indices")

    resh_aux  = input_aux #Reshape((seqlen,), name="resh_aux")(input_aux)

    emb_layer = Embedding(*emb_matrix.shape, input_length=seqlen, 
                          trainable=False, weights=[emb_matrix], 
                          name="embedding")
    emb_x = emb_layer(input_x)
    concat_x = Concatenate(name="concat_x")([emb_x, resh_aux])
    
    yhat = concat_x
    
    for i, layer_size in enumerate(lstm_sizes):
        ret_sequences = (i < len(lstm_sizes)-1)
        layerno = i+1
        yhat = CuDNNLSTM(layer_size, return_sequences=ret_sequences, name=('lstm%d' % layerno))(yhat)
        yhat = BatchNormalization()(yhat)
        yhat = Dropout(dropout_rate)(yhat)
    
    for layer in range(1, dense_layers+1):
        yhat = Dense(300, activation="relu", name=("dense%d" % layer))(yhat)
        yhat = BatchNormalization()(yhat)
        yhat = Dropout(dropout_rate)(yhat)
   
    # len(vocab)+2 is because the zeroth word is for padding
    # and last word is for "unknown" 
    yhat_aux = Dense(aux_dim, activation="sigmoid")(yhat)
    yhat = Dense(len(vocab)+2, activation="linear")(yhat)
        
    #print(input_sample_indices.dtype, input_sample_indices.shape)
    out_train = Lambda(sampling_layer_gather_nd, name="sampling")([yhat, input_sample_indices])
    out_train = Activation('softmax')(out_train)
    out_train = Concatenate(name="concat_out_train")([out_train, yhat_aux])
    
    print(out_train.shape)
    
    out_predict = Activation('softmax')(yhat)
    out_predict = Concatenate(name="concat_out_predict")([out_predict, yhat_aux])
    
    model_train = Model([input_x, input_aux, input_sample_indices], [out_train])
    model_predict = Model([input_x, input_aux], [out_predict])
    
    return model_train, model_predict

In [23]:
lstm_size=256

mtrain, mpredict = make_model(seqlen=seqlen, sample_size=sample_size, emb_matrix=emb_matrix,
                              lstm_sizes=[256,256],
                              dense_size=256, dense_layers=1,
                              dropout_rate=0.0)

InternalError: Dst tensor is not initialized.
	 [[Node: _arg_embedding/Placeholder_0_0/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_6__arg_embedding/Placeholder_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: embedding/Assign/_7 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7_embedding/Assign", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
def mean_pred(y_true, y_pred):
    print(tf.shape(y_true))
    print(tf.shape(y_pred))
    r = tf.multiply(y_true[:,0], tf.round(y_pred[:,0]))
    r = tf.reduce_sum(r, axis=-1)
    r = tf.reduce_mean(r, axis=-1)
    return r

In [14]:
opt = K.optimizers.Adam(lr=0.0005)
mtrain.compile(opt, loss='categorical_crossentropy', metrics=["accuracy"])

In [26]:
imp.reload(util)
batch_size=1024
train_seq = util.NegativeSamplingPermutedSequence(data_x=X, data_xu=Xu,
                                                 seqlen=seqlen, batch_size=batch_size,
                                                 sample_size=sample_size,
                                                 vocab_size=len(vocab)+1)

In [23]:
train_seq[0]

[[array([[0.000e+00, 5.428e+03, 2.371e+03, ..., 2.200e+01, 5.000e+02,
          2.083e+03],
         [5.560e+03, 0.000e+00, 2.726e+03, ..., 4.000e+00, 2.675e+03,
          1.652e+03],
         [0.000e+00, 7.555e+03, 1.000e+01, ..., 2.000e+00, 3.610e+02,
          1.150e+02],
         ...,
         [1.844e+03, 0.000e+00, 1.000e+00, ..., 2.000e+00, 1.298e+03,
          5.390e+03],
         [1.800e+01, 7.000e+00, 0.000e+00, ..., 4.055e+03, 8.850e+02,
          1.000e+00],
         [1.185e+03, 4.000e+00, 2.000e+00, ..., 2.200e+01, 1.460e+02,
          7.610e+02]], dtype=float32), array([[[0, 0],
          [0, 1],
          [0, 1],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
  
         [[0, 0],
          [1, 0],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
  
         [[1, 0],
          [0, 1],
          [0, 0],
          ...,
          [0, 0],
          [0, 0],
          [0, 0]],
  
         ...,
  
         [[0, 1],
   

In [16]:
mtrain.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_x (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 300)      3000300     input_x[0][0]                    
__________________________________________________________________________________________________
input_aux (InputLayer)          (None, 64, 2)        0                                            
__________________________________________________________________________________________________
concat_x (Concatenate)          (None, 64, 302)      0           embedding[0][0]                  
                                                                 input_aux[0][0]                  
__________

In [17]:
print(len(X), seqlen*batch_size)

16213415 65536


In [18]:
checkpoint_filepath = "checkpoints/weights.lstm%d.glove%d.sample%d.hdf5" % (lstm_size, glove_dims, sample_size)

checkpoint = K.callbacks.ModelCheckpoint(checkpoint_filepath,                       
                                        verbose=2,
                                        save_best_only=True,
                                         monitor='loss'
                                        )

In [27]:

numbatches = 2 * X.shape[0] // (batch_size*seqlen)

mtrain.fit_generator(train_seq, steps_per_epoch=numbatches, epochs=5000000,
                    callbacks=[checkpoint],
                    initial_epoch=2)


Epoch 3/5000000


TypeError: 'NegativeSamplingPermutedSequence' object is not an iterator

In [ ]:
mtrain.save("checkpoints/1.hdf5")

### Test

In [43]:
path_misc = "/home/pmilovanov/hg/my/data/gallery-pr/misc"

X_test, Xu_test = util.load_data(path_test, vocab, pad=seqlen, lowercase=True)
#X_test, Xu_test = util.load_data(path_misc, vocab, pad=seqlen, lowercase=True)

100%|##########| 472/472 [00:01<00:00, 434.55it/s]


In [44]:
X_test
print("Unknown words: {:.2f}%".format(np.sum(Xu_test[:,0]) / len(Xu_test)))


Unknown words: 0.08%


In [45]:
mpredict.compile(opt, loss="categorical_crossentropy")

In [46]:
len(words)

10001

In [47]:
def groom_probs(scores, alpha):
    s2 = np.power(scores, alpha)
    total = np.sum(s2)
    return s2 / total


z = groom_probs(np.array([0.6, 0.3, 0.1]),
                  3.0)

print(z)
print(np.sum(z))

[0.8852459  0.11065574 0.00409836]
0.9999999999999999


In [53]:
def capitalize(s):
    if len(s) == 0: return s
    return s[0].upper() + s[1:]



i = np.random.randint(len(X_test) - seqlen)
gen = X_test[i:i+seqlen].tolist()
genu = Xu_test[i:i+seqlen].tolist()


tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen, 2), dtype="float32")
results = []

for i, idx in enumerate(gen):
    word = "<UNK>"
    if genu[i][0] < 0.1:
        word = words[idx]
    if genu[i][1] > 0.9:
        word = capitalize(word)
    sys.stdout.write(word + " ")
    sys.stdout.flush()

print("=" * 100)

UNK_IDX = len(words)


punct = ":-;.,!?'\")"
punct2 = "-/'(\""

iterations = 512

prev_word=words[gen[-1]]
word=""

chars=0
for j in range(iterations):
    tX[0] = np.array(gen[-seqlen:], "int32")
    tXu[0] = np.array(genu[-seqlen:], "float32")
    
    #print(tX)
    z = mpredict.predict([tX, tXu])
    scores=z[0][:-2]
    aux=z[0][-2:]
    idx = UNK_IDX
    #print(scores)
    while idx == UNK_IDX:
        idx = np.random.choice(range(len(vocab)+2), p=groom_probs(scores, 1.0))
    if idx != UNK_IDX:
        gen.append(idx)
        genu.append([0.0, aux[1]])
        word=words[idx]
        if aux[1] > 0.5:
            word = capitalize(word)
        results.append(word)
    else:
        gen.append(0)
        genu.append([1.0, 0.0])
        results.append("<UNK>")
    #print("idx", idx, UNK_IDX)

    if 80-chars < len(word)+1:
        sys.stdout.write("\n")
        chars=0
    if punct.find(word) < 0 and punct2.find(prev_word) < 0:
        sys.stdout.write(" ")
        chars+=1
    sys.stdout.write(word)
    chars+=len(word)
    sys.stdout.flush()
    
    prev_word=word
#    print("%s ", results[-1])

of Desire Machine Collective . Through extended <UNK> by foot or by boat , and accompanied by a shifting ensemble of fellow <UNK> , all four artists work against contemporary notions of immediacy and access in a globalized world where any data point appears to be a search away and other cultures can seem <UNK> with a quick round - trip . This exhibition ====================================================================================================
 gives reflection of museum-based sounds that calls a theatrical play of the
 artist's work. Filming for the project also employs a documentary piece of the
 library. The interplay of mechanisms, can be suggested by a particular number
 of artists these works in various live are often a hybrid yet time of moving
 points. Calarts out towards minimalism and return however, in parallel changes
 to place, materials sculptures of hand, plants possess those attacks, then the
 display, another operated more the purpose of reference, and abstrac

In [39]:
len(X_test)

test_seq = util.NegativeSamplingPermutedSequence(data_x=X_test, data_xu=Xu_test, batch_size=1024, seqlen=seqlen, vocab_size=len(vocab)+1, sample_size=sample_size)

In [40]:
mtrain.evaluate_generator(test_seq, steps=100, verbose=1)

100/100 [==============================] - 4s 42ms/step


[0.9795894759893418, 0.853818359375]

In [41]:
mtrain.evaluate_generator(train_seq, steps=100, verbose=1)

100/100 [==============================] - 4s 40ms/step


[0.9828013104200363, 0.85705078125]

In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'